In [2]:
import tensorflow as tf
import os
import random
import numpy as np

k = 5

positive = os.listdir('kfold_data/positive')
negative = os.listdir('kfold_data/negative')

In [12]:
def getImage(path, size=(200, 200)):
    img = tf.keras.preprocessing.image.load_img(path, target_size=size)
    return tf.keras.preprocessing.image.img_to_array(img)

def getLabel(path):
    if path in positive:
        return 1
    if path in negative:
        return 0

def createModel(optimizer = 'adam', size=(200, 200)):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [13]:
import random
allpaths = positive + negative
random.shuffle(allpaths)

foldamount = len(allpaths) // k

folds = []

for fold in range(k):
    folds.append(allpaths[fold*foldamount:(fold+1)*foldamount + 1])


In [15]:
models = []
foldLabels = []
for fold in folds:
    models.append(createModel())
    foldLabel = []
    for item in fold:
        foldLabel.append(getLabel(item))  
    foldLabels.append(foldLabel)

In [30]:
totalHistory = []
for i in range(k):
    training_images, training_labels, testing_images, testing_labels = [], [], [], []
    for j in range(k):
        if j != i:
            training_images += list(map(lambda x: getImage('alldata/' + x), folds[j]))
            training_labels += foldLabels[j]
        else:
            testing_images += list(map(lambda x: getImage('alldata/' + x), folds[j]))
            testing_labels += foldLabels[j]
    print(f"Training fold {i+1}/{k}")
    training_images = np.array(training_images)
    training_labels = np.array(training_labels)
    testing_images = np.array(testing_images)
    testing_labels = np.array(testing_labels)
    
    history = models[i].fit(training_images, training_labels, epochs=10, validation_data=(testing_images, testing_labels), shuffle=True, verbose=1)
    totalHistory.append(history)

Training fold 1/5
Epoch 1/10
7/7 [==============================] - 16s 2s/step - loss: 710.4727 - accuracy: 0.4950 - val_loss: 4.0824 - val_accuracy: 0.4600
Epoch 2/10
7/7 [==============================] - 16s 2s/step - loss: 0.9897 - accuracy: 0.7300 - val_loss: 0.4259 - val_accuracy: 0.8200
Epoch 3/10
7/7 [==============================] - 13s 2s/step - loss: 0.3993 - accuracy: 0.8550 - val_loss: 0.5050 - val_accuracy: 0.7400
Epoch 4/10
7/7 [==============================] - 17s 2s/step - loss: 0.3257 - accuracy: 0.8700 - val_loss: 0.4591 - val_accuracy: 0.8000
Epoch 5/10
7/7 [==============================] - 11s 2s/step - loss: 0.1853 - accuracy: 0.9500 - val_loss: 0.5775 - val_accuracy: 0.8400
Epoch 6/10
7/7 [==============================] - 18s 3s/step - loss: 0.0784 - accuracy: 0.9800 - val_loss: 0.6056 - val_accuracy: 0.8600
Epoch 7/10
7/7 [==============================] - 12s 2s/step - loss: 0.0707 - accuracy: 0.9800 - val_loss: 0.3595 - val_accuracy: 0.8400
Epoch 8/10
7/7

In [55]:
for i in range(k):
    models[i].save(f'fold_models/fold_model{i + 1}.h5')

In [61]:
for history in totalHistory:
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    print(val_acc[len(val_acc) - 1])

0.8399999737739563
0.699999988079071
0.8600000143051147
0.8199999928474426
0.7200000286102295
